In [1]:
import requests
import json
import time
import re
import random
import pymongo
import pandas as pd

In [2]:
def tratamento(t):
        #remover os @ marcados no tweet
        t=re.sub(r'@[aA-zZ]* ', '', t, flags=re.MULTILINE)

        #remover dados quando é RT
        t=re.sub(r'RT @*[aA-zZ]*[0-9]*: ', '', t, flags=re.MULTILINE)

        #remover links
        t=re.sub(r'https:\/\/t.co\/*[aA0-zZ9]* ', '', t, flags=re.MULTILINE)
        
        #remover hashtags
        t=re.sub(r'#[aA-zZ]*[0-9]*', '', t, flags=re.MULTILINE)

        emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)

        t=emoji_pattern.sub(r' ', t)


        return t

In [3]:
def connection(name_bd):
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient[name_bd]
    return mydb

In [4]:
#buscar uma collection especifica informando o nome da collection e do banco
#[retorno é um dicionário com a collection]
def get_collection(name_collection, mydb):
    collection = mydb[name_collection]
    return collection

In [5]:
#retorna uma lista de dicionarios
#cada dicionario tem a forma {'tema':'nome_tema', 'tweets': []}
#em tweets sera uma lista de tweets sobre cada tema presente na collection

#recebe como parametro uma collection
def get_all_tweets(collection):
    l_tweets=[]
    for doc in collection.find():    
        j=0 #controla cada tema da collection

        i=0 #controla cada tweet sobre o tema

        #criar o dicionario para o tema
        dic={'tema':doc['tema'],'tweets':[]}
        while(i<len(doc['tweet'])):
            dic['tweets'].append(doc['tweet'][i]['texto'])
            i=i+1

        l_tweets.append(dic)
        j=j+1
    
    return l_tweets

In [6]:
#coletar o tweet do banco
banco=connection('dashboard')
#nome de todas as collections do bd
all_NamesCollections = banco.list_collection_names()
i=0
l_collection=[] #lista de collections
while(i<3):
    x = random.randint(0,len(all_NamesCollections))
    l_collection.append(get_collection(all_NamesCollections[x],banco))
    i=i+1

In [7]:
df = pd.DataFrame(columns=["tema","texto"])
df

,tema,texto


In [8]:
#coletar o tweet do banco

collection = get_collection('10/05/2021, 00:16:10',banco)

df = pd.DataFrame(columns=["tema","texto"])

#coletar os tweets da base pra poder gerar o df
k=0
lista = get_all_tweets( collection )
        
p=0
while(p<len(lista)):
    for tweet in lista[p]['tweets']:
        df.loc[len(df)]=[ lista[p]['tema'], tratamento(tweet)]
                
    p+=1
k+=1

df.head()

,tema,texto
0,#ProvaDeFogo,Érika protagonista. Érika injustiçada! Pode en...
1,#ProvaDeFogo,"A Tati falando que a Érika não joga bem, Imagi..."
2,#ProvaDeFogo,Eliminem a Tati para ela ficar com o Mussunzin...
3,#ProvaDeFogo,O diálogo da briga do Rico e Solange sempre no...
4,#ProvaDeFogo,a adriane nem esconde pra quem estava torcendo...


In [9]:
#coletar os tweets da base pra poder gerar a base de testes
k=0
while(k<len(l_collection)):
    lista = get_all_tweets( l_collection[k] )
    
    p=0
    while(p<len(lista)):
        for tweet in lista[p]['tweets']:
            df.loc[len(df)]=[ lista[p]['tema'], tratamento(tweet)]
            
        p+=1
    k+=1

In [146]:
df.to_csv('dados_testes.csv')

Aqui começa os passos para obter a análise de sentimentos do tema

In [10]:
#coletar o nome das ultimas 10 collections
all_NamesCollections = banco.list_collection_names()
all_NamesCollections.sort()

#nomes das ultimas collections
ultimos=[]
i=1
while(i<10):
    ultimos.append( all_NamesCollections[len(all_NamesCollections)-i])
    i+=1
ultimos

i=0
l_collection=[] #lista com obj collections
while(i<len(ultimos)):
    l_collection.append(get_collection(ultimos[i],banco))
    i+=1

In [11]:
#cria um dataframe de todas as collections juntas
df2 = pd.DataFrame(columns=["tema","texto"])
#coletar os tweets da base pra poder gerar o df
k=0
while(k<len(l_collection)):
    lista = get_all_tweets( l_collection[k] )
    
    p=0
    while(p<len(lista)):
        for tweet in lista[p]['tweets']:
            df2.loc[len(df2)]=[ lista[p]['tema'], tratamento(tweet)]
            
        p+=1
    k+=1

In [12]:
#conta quantos temas tem no dataframe
temas = df2['tema'].value_counts()

In [13]:
#cria um dicionario com os temas e quantos tweets ele tem
#cria uma lista com os nomes dos temas
dic_temas  = dict(temas)
nome_temas = list(dic_temas.keys())

In [14]:
#removendo temas que tem menos de 100 tweets coletados
i=0
while(i<len(nome_temas)):
    if(dic_temas[nome_temas[i]] < 100):
        del dic_temas[nome_temas[i]]
    i+=1

In [15]:
s = 'Venezuela'

In [16]:
#separa do datraframe gigante apenas as linhas que tem um tema especifico
df_de_um_tema = df2.query('tema == "'+ s +'"')
df_de_um_tema.head()

,tema,texto
19931,Venezuela,O q eu quero hoje \n\nGol do Gabigol assistênc...
19932,Venezuela,Empatou Ag 1 x 1.....\nAinda tem gente que\nFa...
19933,Venezuela,Gol do Brasil - Marquinhos \nVenezuela 1x1 Bra...
19934,Venezuela,● ELIMINATÓRIAS COPA DO MUNDO ●\nGOOOOOOOOOOOO...
19935,Venezuela,"Venezuela tomou Gol, agora eles querem apelar ..."


In [23]:
#verifica se o sentimento daquele tema é negativo ou positivo

#processa os dados para passar pro modelo
tweets = df_de_um_tema['texto']
tweets = [Preprocessing(i) for i in tweets]
tweets_testes = vectorizer.transform(tweets)

# 1= negativo
# 0= positivo
positivo=0
negativo=0

#conta qual sentimento mais aparece
for t, c in zip (tweets,modelo.predict(tweets_testes)):
    if(c==1):
        negativo+=1
    else:
        positivo+=1
        
if(negativo>positivo):
    print("Negativo")
else:
    print("Positivo")

NameError: name 'nltk' is not defined